In [7]:
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset

from torchvision import transforms, models
from PIL import Image

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

Load Raw Data & Match with Available Images

In [9]:
BASE_DIR = Path.cwd().parent if "notebooks" in str(Path.cwd()) else Path.cwd()

raw_df = pd.read_csv("/content/drive/MyDrive/satellite-property-valuation/data/raw/train_data.csv")
raw_df["id"] = raw_df["id"].astype(int)

IMG_DIR = "/content/drive/MyDrive/satellite-property-valuation/data/images"


In [11]:
available_ids = sorted(
    int(float(p.stem)) for p in Path(IMG_DIR).glob("*.png")
)

fusion_df = (
    raw_df[raw_df["id"].isin(available_ids)]
    .drop_duplicates("id")
    .sort_values("id")
    .reset_index(drop=True)
)

assert len(fusion_df) == len(available_ids)

In [12]:
fusion_df.shape


(5488, 21)

Resolve Image Paths

In [16]:
img_paths = []

# Convert IMG_DIR to a Path object to enable path joining with the / operator
img_dir_path_obj = Path(IMG_DIR)

for pid in fusion_df["id"].values:
    p1 = img_dir_path_obj / f"{pid}.0.png"
    p2 = img_dir_path_obj / f"{pid}.png"
    img_paths.append(p1 if p1.exists() else p2)

len(img_paths), img_paths[:3]

(5488,
 [PosixPath('/content/drive/MyDrive/satellite-property-valuation/data/images/1200019.0.png'),
  PosixPath('/content/drive/MyDrive/satellite-property-valuation/data/images/1200021.0.png'),
  PosixPath('/content/drive/MyDrive/satellite-property-valuation/data/images/3600057.0.png')])

Image Transform & Dataset

In [17]:
img_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
])


In [18]:
class ImageOnlyDataset(Dataset):
    def __init__(self, paths, transform):
        self.paths = paths
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        return self.transform(img)


CNN Backbone for Image Embeddings

In [19]:
cnn = models.resnet18(
    weights=models.ResNet18_Weights.IMAGENET1K_V1
)
cnn.fc = nn.Identity()

for param in cnn.parameters():
    param.requires_grad = False

cnn = cnn.to(device)
cnn.eval()


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 60.7MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Extract Image Features

In [20]:
img_dataset = ImageOnlyDataset(img_paths, img_tfms)
img_loader = DataLoader(
    img_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0
)


In [21]:
img_features = []

with torch.no_grad():
    for batch in img_loader:
        batch = batch.to(device)
        feats = cnn(batch)
        img_features.append(feats.cpu().numpy())

X_img = np.vstack(img_features)
X_img.shape


(5488, 512)

In [22]:
np.save(
    "/content/drive/MyDrive/satellite-property-valuation/data/processed/image_embeddings_fusion.npy",
    X_img
)


Prepare Tabular Features

In [23]:
target_col = "price"
drop_cols = ["id", "date", target_col]

X_tab = fusion_df.drop(columns=drop_cols)
y = np.log1p(fusion_df[target_col].values)


In [24]:
scaler = StandardScaler()
X_tab_scaled = scaler.fit_transform(X_tab)

X_tab_scaled.shape, y.shape


((5488, 18), (5488,))

Early Fusion (Concatenation)

In [25]:
X_fused = np.concatenate([X_tab_scaled, X_img], axis=1)
X_fused.shape


(5488, 530)

Train–Validation Split

In [26]:
X_tr, X_va, y_tr, y_va = train_test_split(
    X_fused,
    y,
    test_size=0.2,
    random_state=42
)


Torch Datasets

In [27]:
X_tr_t = torch.tensor(X_tr, dtype=torch.float32)
y_tr_t = torch.tensor(y_tr, dtype=torch.float32).unsqueeze(1)

X_va_t = torch.tensor(X_va, dtype=torch.float32)
y_va_t = torch.tensor(y_va, dtype=torch.float32).unsqueeze(1)


In [28]:
train_dl = DataLoader(
    TensorDataset(X_tr_t, y_tr_t),
    batch_size=64,
    shuffle=True
)

val_dl = DataLoader(
    TensorDataset(X_va_t, y_va_t),
    batch_size=64,
    shuffle=False
)


Fusion Regressor Network

In [29]:
class FusionRegressor(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)


In [30]:
fusion_model = FusionRegressor(X_tr.shape[1]).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(
    fusion_model.parameters(),
    lr=3e-4
)


Training Loop

In [31]:
epochs = 15

for ep in range(epochs):
    fusion_model.train()
    train_loss = 0.0

    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        out = fusion_model(xb)
        loss = loss_fn(out, yb)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * xb.size(0)

    train_loss /= len(train_dl.dataset)

    fusion_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = fusion_model(xb)
            val_loss += loss_fn(preds, yb).item() * xb.size(0)

    val_loss /= len(val_dl.dataset)

    print(
        f"Epoch {ep+1}/{epochs} | "
        f"Train MSE: {train_loss:.4f} | "
        f"Val MSE: {val_loss:.4f}"
    )


Epoch 1/15 | Train MSE: 42.3523 | Val MSE: 3.8176
Epoch 2/15 | Train MSE: 3.0665 | Val MSE: 2.2683
Epoch 3/15 | Train MSE: 1.7025 | Val MSE: 1.2625
Epoch 4/15 | Train MSE: 1.0417 | Val MSE: 0.8589
Epoch 5/15 | Train MSE: 0.7293 | Val MSE: 0.6439
Epoch 6/15 | Train MSE: 0.5832 | Val MSE: 0.5503
Epoch 7/15 | Train MSE: 0.5024 | Val MSE: 0.4998
Epoch 8/15 | Train MSE: 0.4534 | Val MSE: 0.5106
Epoch 9/15 | Train MSE: 0.4216 | Val MSE: 0.4406
Epoch 10/15 | Train MSE: 0.3901 | Val MSE: 0.4263
Epoch 11/15 | Train MSE: 0.3722 | Val MSE: 0.4164
Epoch 12/15 | Train MSE: 0.3548 | Val MSE: 0.3953
Epoch 13/15 | Train MSE: 0.3432 | Val MSE: 0.3849
Epoch 14/15 | Train MSE: 0.3321 | Val MSE: 0.3750
Epoch 15/15 | Train MSE: 0.3141 | Val MSE: 0.3662


Final Evaluation

In [32]:
fusion_model.eval()

with torch.no_grad():
    preds_log = fusion_model(X_va_t.to(device)).cpu().numpy().ravel()

pred_price = np.expm1(preds_log)
true_price = np.expm1(y_va)


In [33]:
rmse = root_mean_squared_error(true_price, pred_price)
r2 = r2_score(true_price, pred_price)

print(f"RMSE: {rmse:.2f}")
print(f"R²  : {r2:.4f}")


RMSE: 489662.99
R²  : -0.6775
